In [ ]:
import pandas as pd
import requests
from numpy import nan
from time import sleep
import numpy as np

api_cep = "https://brasilapi.com.br/api/cep/v1/CEP"
arquivo = ''

In [ ]:
def pesquisaCEP(linhaBase):
    numero_str = str(linhaBase['cep'])
    numero_formatado = numero_str.zfill(8)
    r = requests.get(api_cep.replace("CEP",numero_formatado))
    if r.status_code == 200:
        try:
            ##return r.json()["city"], r.json()["state"], r.json()["neighborhood"]
            return pd.Series({'localidade': r.json()["city"], 'uf': r.json()["state"], 'bairro': r.json()["neighborhood"]})

        except KeyError:
          print(numero_formatado)
          return pd.Series({'localidade': nan, 'uf': nan, 'bairro': nan})
    else: return pd.Series({'localidade': nan, 'uf': nan, 'bairro': nan})

In [ ]:
df_consolidado = pd.read_excel(arquivo, sheet_name='Sheet1')
df_consolidado = df_consolidado.replace(nan,0)
df_consolidado = df_consolidado.astype({'cep': 'Int64'})

In [ ]:
df_split = np.array_split(df_consolidado,10)

In [ ]:
i = 1

for df in df_split:
    print(f"{i}/10")
    df[['localidade','uf','bairro']] = df.apply(pesquisaCEP, axis=1, result_type='expand')
    i += 1

In [ ]:
df_com_localidades = pd.concat(df_split)

In [ ]:
df_com_localidades.to_excel("consolidado_inscricoes_jan-2016_dez-2023_com_bairro.xlsx",index=False)